In [18]:
# !pip install turtle
# !pip install scipy
# !pip install ghostscript
# !pip install pillow
# !conda install numpy
# # !conda update -n base -c defaults conda

In [19]:
import random
import math
import os
import numpy as np
from PIL import Image
from turtle import *


In [20]:
def shitty_rainbow(red, green, blue, step=5):
        if red > 0 and green < 256 and blue == 0:
            red -= step
            green += step
            if red < step:
                red = 0
        if green > 0 and blue < 256 and red == 0:
            green -= step
            blue += step
            if green < step:
                green = 0
        if blue > 0 and red < 256 and green == 0:
            blue -= step
            red += step
            if blue < step:
                blue = 0

        return [red, green, blue]

def good_rainbow(color1, color2, step):
    r = int((1 - step) * color1[0] + step * color2[0])
    g = int((1 - step) * color1[1] + step * color2[1])
    b = int((1 - step) * color1[2] + step * color2[2])
    return r, g, b

In [21]:
class Color:
    def __init__(self, red=0, green=0, blue=0):
        self.r = red % 256
        self.g = green % 256
        self.b = blue % 256

    def get_colors(self):
        self.update_color()
        return self.r, self.g, self.b

    def set_colors(self, red, green, blue):
        self.r = red % 256
        self.g = green % 256
        self.b = blue % 256

    def update_color(self, f=lambda x:x):
        self.set_colors(f(self.r), f(self.g), f(self.b))

class Same_color(Color):
    def __init__(self, red=0, green=0, blue=0):
        super().__init__(red, green, blue)

    def get_colors(self):
        return super().get_colors()

    def set_colors(self, red, green, blue):
        return super().set_colors(red, green, blue)

    def update_color(self):
        new_color = [self.r, self.g, self.b]
        self.set_colors(*new_color)

class Rainbow(Color):
    def __init__(self, red=0, green=0, blue=0):
        super().__init__(red, green, blue)

    def get_colors(self):
        return super().get_colors()

    def set_colors(self, red, green, blue):
        return super().set_colors(red, green, blue)

    def update_color(self):
        new_color = shitty_rainbow(self.r, self.g, self.b)
        self.set_colors(*new_color)

class Good_rainbow(Color):
    colors = [(128, 128, 200), (31, 216, 205), (127, 221, 115),
              (254, 186, 0), (255, 93, 85), (255, 0, 170)]
    index = 0
    count = 0
    def __init__(self, red=0, green=0, blue=0, step = 10):
        self.step = step
        super().__init__(red, green, blue)

    def get_colors(self):
        return super().get_colors()

    def set_colors(self, red, green, blue):
        return super().set_colors(red, green, blue)

    def update_color(self):
        color2 = self.colors[self.index]
        color1 = self.colors[self.index + 1 if self.index <= 4 else 0]
        self.count += 1
        if self.count > self.step:
            self.count = 0
            self.index = self.index + 1 if self.index <= 4 else 0
        new_color = good_rainbow(color1, color2, 0 if self.count == 0 else 1 / self.count)
        self.set_colors(*new_color)

class Cherry(Color):
    max_depth = -1
    flowers = [(222, 136, 149), (240, 168, 179), (241, 176, 186), (243, 190, 198), (246, 206, 212), (248, 214, 220)]
    bark = [(86, 73, 78), (96, 69, 76), (106, 65, 73)]

    def __init__(self, red=0, green=0, blue=0, depth=10):
        super().__init__(red, green, blue)
        depth = depth

    def get_colors(self, depth):
        proportion = depth / self.max_depth
        if proportion > 0.6:
            choice = random.choices(self.bark, weights=[2, 1, 1])[0]
        elif proportion > 0.5:
            choice = random.choices(self.bark + self.flowers, weights=[1, 3, 3, 2, 1, 1, 1, 1, 1], k=1)[0]
        else:
            choice = random.choices(self.flowers, weights=[1, 1, 2, 2, 3, 3], k=1)[0]



        red = choice[0]
        green = choice[1]
        blue = choice[2]
        return red, green, blue

In [22]:
default_palette = Color(240, 168, 179)
rainbow_palette = Rainbow(0,0,255)
good_rainbow_palette = Good_rainbow(step=10)
cherry_palette = Cherry()
one_color_palette = Same_color(255, 255, 255)


In [23]:
def dandelion(centre=(0,0), steps=27, length=70, palette=default_palette):
    dgr = 360 / steps
    for i in range(steps):
        up()
        goto(centre)
        down()
        color(palette.get_colors())
        left(dgr)
        forward(length)



In [24]:
def flower_power(start, length_inner, length_outer, precision, palette, count=None):
    color(palette.get_colors())
    if count == None:
        flower_power(start, length_inner, length_outer, precision, palette, length_inner / (length_outer - length_inner))
    elif count > 0:
        fillcolor(palette.get_colors())
        begin_fill()
        up()
        goto(start)
        down()

        points_inner = [(length_inner * math.cos(tetha),
                length_inner * math.sin(tetha))
                for tetha in [math.pi * 2 * x / precision for x in range(1, precision + 1)]]
        points_outer = [(length_outer * math.cos(tetha),
                length_outer * math.sin(tetha))
                for tetha in [math.pi * 2 * x / precision for x in range(1, precision + 1)]]


        for i in list(range(len(points_inner))):
            up()
            goto(points_inner[i])
            down()
            goto(points_outer[i])

            i_next = i + 1 if i + 1 < len(points_inner) else 0

            r = math.sqrt((points_outer[i][0] - points_outer[i_next][0]) ** 2 +
                    (points_outer[i][1] - points_outer[i_next][1]) ** 2) / 2

            tetha = math.atan2(points_outer[i][1], points_outer[i][0]) - math.pi / 2 + math.pi / 8
            tetha_next = tetha + math.pi


            curve = [(r * math.cos(angle) + (points_outer[i][0] + points_outer[i_next][0]) / 2,
                r * math.sin(angle) + (points_outer[i][1] + points_outer[i_next][1]) / 2)
                for angle in [(tetha * (10 - x) + tetha_next * x) / 10 for x in range(0, 11)]]

            for point in curve:
                goto(point)

        end_fill()
        flower_power(start, length_inner - (length_outer - length_inner), length_inner, precision, palette, count - 1)


In [25]:
def tree(start, length, angle, depth, palette, noise=True, noise_amount=0.1, complemantary=True):
    if type(palette).__name__ == Cherry.__name__:
        color(palette.get_colors(depth))
    else:
        color(palette.get_colors())
    up()
    goto(start)
    left(90)
    down()
    width(depth + 2)
    noise = random.random() * length * noise_amount
    forward(length + noise)
    # turtle.setheading(0)
    new_start = pos()
    noise = random.random() * angle * noise_amount
    draw_branch(new_start, length * 0.7, angle + noise, 0, depth - 1, palette, noise)
    seth(90)
    noise = random.random() * angle * noise_amount
    if complemantary:
        draw_branch(new_start, length * 0.7, 90 - angle + noise, 1, depth - 1, palette, noise)
    else:
        draw_branch(new_start, length * 0.7, angle + noise, 1, depth - 1, palette, noise)

def draw_branch(start, length, angle, direction, depth, palette, noise=True,noise_amount=0.1, complemantary=True):
    if depth == 0:
        return
    width(depth + 2)
    if type(palette).__name__ == Cherry.__name__:
        color(palette.get_colors(depth))
    else:
        color(palette.get_colors())
    goto(start)
    down()
    if direction == 0:
        left(angle)
    else:
        right(angle)
    noise = random.random() * length * noise_amount
    forward(length + noise)
    new_start = pos()
    new_angle = heading()
    up()
    noise = random.random() * angle * noise_amount
    draw_branch(new_start, length * 0.7, angle + noise, 0, depth - 1, palette, noise)
    seth(new_angle)
    noise = random.random() * angle * noise_amount

    if complemantary:
        draw_branch(new_start, length * 0.7, 90 - angle + noise, 1, depth - 1, palette, noise)
    else:
        draw_branch(new_start, length * 0.7, angle + noise, 1, depth - 1, palette, noise)


In [26]:
def triunghiul_psihedelic(start, length, palette=default_palette):
    color(palette.get_colors())
    up()
    goto(start)

    left(60)
    down()
    triunghi_go_brr(length, 0, palette)

def triunghi_go_brr(length, count, palette=default_palette):
    color(palette.get_colors())
    forward(length)

    if count < 105:
        left(120 * 0.995)
        triunghi_go_brr(length * 0.98267, count + 1, palette)

random dandelions

In [ ]:
dandelion_steps = 27
dandelion_length = 100
dandelion_number = 50
file_name = 'dandelions.ps'

In [ ]:
(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
for _ in range(dandelion_number):
    # dandelion((random.randint(-screen_width, screen_width), random.randint(-screen_heigth, screen_heigth)),
    #            dandelion_steps, dandelion_length, rainbow_palette)
    dandelion((random.randint(-screen_width, screen_width), random.randint(-screen_heigth, screen_heigth)),
               dandelion_steps, dandelion_length, good_rainbow_palette)

canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()

full random flowers

In [ ]:
flower_precision = 75
flower_length_inner = 765
flower_length_outer = 790
flower_count = None
file_name = 'output/flower_power_2.ps'

In [ ]:
(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
# flower_power((0,0), flower_length_inner, flower_length_outer, flower_precision, rainbow_palette)
flower_power((0,0), flower_length_inner, flower_length_outer, flower_precision, good_rainbow_palette)

canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()

fractal rainbow tree with no noise and perfect symmetry 

In [ ]:
file_name = 'output/tree_no_noise_symmetry.ps'

(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
# tree((0, -250), 150, 45, 11, rainbow_palette, noise_amount=0)
tree((0, -250), 150, 45, 11, good_rainbow_palette, noise_amount=0)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


fractal rainbow tree with no noise and no symmetry 

In [ ]:
file_name = 'output/tree_no_noise_no_symmetry.ps'

(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
# tree((0, -250), 150, 30, 11, rainbow_palette, noise_amount=0)
tree((0, -250), 150, 30, 11, good_rainbow_palette, noise_amount=0)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


fractal rainbow tree with noise and no symmetry 

In [ ]:
file_name = 'output/tree_noise_no_symmetry.ps'

(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
# tree((0, -250), 150, 30, 11, rainbow_palette, noise_amount=0.3)
tree((0, -250), 150, 30, 11, good_rainbow_palette, noise_amount=0.3)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


fractal rainbow tree with noise and no symmetry no complementary angles

In [ ]:
file_name = 'output/tree_noise_no_symmetry_no_comp.ps'

(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
# tree((0, -250), 150, 30, 11, rainbow_palette, noise_amount=0.3, complemantary=False)
tree((0, -250), 150, 30, 11, good_rainbow_palette, noise_amount=0.3, complemantary=False)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


fractal CHERRY BLOSSOM tree with noise and no symmetry 

In [ ]:
file_name = 'output/CHERRY_tree_noise_no_symmetry.ps'
cherry_palette.max_depth = 11
(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
tree((0, -250), 150, 30, 11, cherry_palette, noise_amount=0.3)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


triangle

In [ ]:
file_name = 'output/triangle.ps'

(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(10)
speed(1000)
hideturtle()
triunghiul_psihedelic((0,-300), 700, good_rainbow_palette)
canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()


composition of flowers, triangle, dandelions and tree

In [36]:
file_name = 'output/composition_2.ps'


(screen_width, screen_heigth) = screensize()
colormode(cmode=255)
width(3)
speed(1000)
hideturtle()
flower_power((0,0), 765, 820, 50, good_rainbow_palette)
for _ in range(30):
    dandelion((random.randint(-screen_width, screen_width), random.randint(-screen_heigth, screen_heigth)),
               30, 150, one_color_palette)
width(9)
triunghiul_psihedelic((0,-300), 700, one_color_palette)

tree((0, -250), 150, 30, 11, good_rainbow_palette, noise_amount=0.9)

canvas = getcanvas()

if canvas is None:
    print("Failed to retrieve canvas object")

canvas.postscript(file=file_name)
exitonclick()

Convert all images to pngs

In [45]:
to_convert = []
for file in os.listdir('output/'):
    if '.ps' in file:
        to_convert.append('output/' + file)

for file in to_convert:
    print(file[:-3]+ ".png")
    img = Image.open(file)
    img.save(file[:-3] + ".png")

output/composition.png
output/composition_2.png
